# Tarea 01
## Teoria de la computacion
![img](https://imgs.search.brave.com/IGJUOrKWTz-bnFOL4hGIv6Id086V-tWx4rqhK79G2yU/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly93YWxs/cGFwZXJzLmNvbS9p/bWFnZXMvZmVhdHVy/ZWQvc29sby1sZXZl/bGluZy00ay1yMHg3/MXFzeG51eGU3Z3pv/LmpwZw)

In [99]:
# Clase RE que representa una expresión regular
from abc import ABC, abstractmethod
from typing import Dict, Set, List, Union,Optional

# Clase Symbol que representa un símbolo ASCII o ε
class Symbol:
    def __init__(self, char: str):
        if char == "ε" or (len(char) == 1 and ord(char) < 128):
            self.char = char
        else:
            raise ValueError("El símbolo debe ser un carácter ASCII o ε")

    def __eq__(self, other):
        return isinstance(other, Symbol) and self.char == other.char

    def __lt__(self, other):
        return isinstance(other, Symbol) and self.char < other.char

    def __str__(self):
        return self.char
class Symbol:
    def __init__(self, value: str):
        self.value = value

    def __str__(self):
        return self.value

    def __eq__(self, other):
        if isinstance(other, Symbol):
            return self.value == other.value
        return False

    def __hash__(self):
        return hash(self.value)

# Clase Alphabet que representa un alfabeto
class Alphabet:
    def __init__(self, symbols=None):
        if symbols is None:
            symbols = set()
        self.symbols = set(symbols)

    def add_symbol(self, symbol: Symbol):
        self.symbols.add(symbol)

    def __contains__(self, symbol):
        return symbol in self.symbols

    def __str__(self):
        return "{" + ", ".join(str(s) for s in self.symbols) + "}"

# Clase Word que representa una cadena de símbolos
class Word:
    def __init__(self, symbols=None):
        if symbols is None:
            symbols = []
        self.symbols = symbols

    def __eq__(self, other):
        return isinstance(other, Word) and self.symbols == other.symbols

    def __str__(self):
        if not self.symbols:
            return "ε"
        return "".join(str(symbol) for symbol in self.symbols)

# Clase Language que representa un lenguaje 
class Language:
    def __init__(self, words=None):
        if words is None:
            words = set()
        self.words = set(words)

    def add_word(self, word: Word):
        self.words.add(word)

    def __contains__(self, word):
        return word in self.words

    def __str__(self):
        return "{" + ", ".join(str(w) for w in self.words) + "}"



# Clase abstracta base para todas las expresiones regulares
class RE(ABC):
    @abstractmethod
    def __str__(self):
        pass

# Clase que representa la expresión regular vacía (Empty)
class Empty(RE):
    def __str__(self):
        return "∅"

# Clase que representa el lenguaje {ε} (Epsilon)
class Epsilon(RE):
    def __str__(self):
        return "ε"

# Clase que representa un símbolo ASCII (Sym)
class Sym(RE):
    def __init__(self, char: str):
        self.char = char

    def __str__(self):
        return self.char

# Clase que representa la concatenación de dos expresiones regulares (Conc)
class Conc(RE):
    def __init__(self, e1: RE, e2: RE):
        self.e1 = e1
        self.e2 = e2

    def __str__(self):
        return f"({self.e1} . {self.e2})"

# Clase que representa la unión de dos expresiones regulares 
class Union(RE):
    def __init__(self, e1: RE, e2: RE):
        self.e1 = e1
        self.e2 = e2

    def __str__(self):
        return f"({self.e1} + {self.e2})"

# Clase que representa la estrella de Kleene de una expresión regular 
class Kleene(RE):
    def __init__(self, e: RE):
        self.e = e

    def __str__(self):
        return f"({self.e})*"

# Clase AutomataFinito como base para AFD y AFNE


# Clase para representar un símbolo o una transición epsilon
class CharWithEpsilon:
    def __init__(self, char: Optional[str] = None):
        self.char = char  # Si char es None, representa una transición epsilon

    def __str__(self):
        return self.char if self.char else 'ε'

    def __eq__(self, other):
        return isinstance(other, CharWithEpsilon) and self.char == other.char

    def __hash__(self):
        return hash(self.char)

# Clase base para autómatas finitos
class AutomataFinito:
    def __init__(self, states=None, init_state=None, final_states=None):
        self.states = states if states is not None else []  # Lista de estados
        self.init_state = init_state if init_state is not None else 0  # Estado inicial
        self.final_states = final_states if final_states is not None else set()  # Conjunto de estados finales
        self.delta = {}  # Diccionario para la función de transición

    def set_states(self, states):
        self.states = states

    def set_init_state(self, init_state):
        self.init_state = init_state

    def set_final_states(self, final_states):
        self.final_states = final_states

# Clase DFA que representa un autómata finito determinista
class DFA(AutomataFinito):
    def __init__(self, states=None, init_state=None, final_states=None):
        super().__init__(states, init_state, final_states)
        self.delta = {}  # Dict[str, Dict[Symbol, str]] para la función de transición

    def set_delta(self, delta: Dict[str, Dict[str, str]]):
        self.delta = delta

    def add_transition(self, from_state: str, symbol: str, to_state: str):
        if from_state not in self.delta:
            self.delta[from_state] = {}
        self.delta[from_state][symbol] = to_state

# Clase NFAE que representa un autómata finito no determinista con transiciones epsilon
class NFAE(AutomataFinito):
    def __init__(self, states=None, init_state=None, final_states=None):
        super().__init__(states, init_state, final_states)
        self.delta = {}  # Dict[str, Dict[CharWithEpsilon, Set[str]]] para la función de transición

    def set_delta(self, delta: Dict[str, Dict[CharWithEpsilon, Set[str]]]):
        self.delta = delta

    def add_transition(self, from_state: str, symbol: CharWithEpsilon, to_states: Set[str]):
        if from_state not in self.delta:
            self.delta[from_state] = {}
        if symbol not in self.delta[from_state]:
            self.delta[from_state][symbol] = set()
        self.delta[from_state][symbol].update(to_states)

    def has_epsilon_transitions(self, estado: str) -> bool:
        """Verifica si el estado actual puede transicionar con epsilon."""
        return CharWithEpsilon(None) in self.delta.get(estado, {})

## First

In [100]:
#funcion que debuelve el simbolo mas a la derecha. En caso de que no haya simbolos, devuelve \epsilon
def first(word: str) -> str:
    if not word:
        return Symbol('ε')
    return word[-1]
#devuelve la subcadena sin el simbolo mas a la derecha
print(first('abn'))

n


## Prefix

In [101]:
def prefix(word: Word) -> Word:
    if not word.symbols:  
        return Word([Symbol("ε")])
    return Word(word.symbols[:-1])  

print(prefix(Word([Symbol('a'), Symbol('b'), Symbol('c')])))

ab


## Membership

In [102]:
#indica si cadena esta en un lenguaje
def membership(Word: Word, Language: Language)->bool:
    return str(Word)in Language
a=Symbol('a') 
b=Symbol('b')
palabra1=Word([a,b])
lenguaje = Language(str(palabra1))
print(str(palabra1)+" esta en el lenguaje"+str(lenguaje)+"?")
print(membership(palabra1,lenguaje))

ab esta en el lenguaje{a, b}?
False


## Clousure

In [103]:
#clausura dado un epsilon automata y un estado regresa la cerradura de epsilon de ese estado
def clousure(nfae: NFAE, estado: str) -> Set[str]:
    stack = [estado]
    closure = set()
    while stack:
        estado_actual = stack.pop()
        closure.add(estado_actual)
        if CharWithEpsilon(None) in nfae.delta.get(estado_actual, {}):
            for siguiente_estado in nfae.delta[estado_actual][CharWithEpsilon(None)]:
                if siguiente_estado not in closure:
                    stack.append(siguiente_estado)
    return closure
#ejemplo de uso
print("ejemplo de clausura")
nfae = NFAE()
nfae.set_delta({
    'q0': {
        CharWithEpsilon('a'): {'q1'},
        CharWithEpsilon(None): {'q2'}
    },
    'q1': {
        CharWithEpsilon('b'): {'q2'}
    }
})
print(clousure(nfae, 'q0')) 
print(clousure(nfae, 'q1'))  

ejemplo de clausura
{'q2', 'q0'}
{'q1'}


## Accept

In [104]:
#accept en esta se recibe una cadena y un epsilon automata y regresa si la cadena es aceptada por el automata o no
def accept(nfae: NFAE, word: Word) -> bool:
    estados_actuales = clousure(nfae, nfae.init_state)
    for symbol in word.symbols:
        siguiente_estado = set()
        for estado in estados_actuales:
            if CharWithEpsilon(str(symbol)) in nfae.delta.get(estado, {}):
                siguiente_estado.update(nfae.delta[estado][CharWithEpsilon(str(symbol))])
        estados_actuales = set()
        for estado in siguiente_estado:
            estados_actuales.update(clousure(nfae, estado))
    return bool(estados_actuales & nfae.final_states)
#ejemplo de uso
print("ejemplo de accept")
nfae = NFAE()
nfae.set_delta({
    'q0': {
        CharWithEpsilon('a'): {'q1'},
        CharWithEpsilon(None): {'q2'}
    },
    'q1': {
        CharWithEpsilon('b'): {'q2'}
    }
})
nfae.set_init_state('q0')
nfae.set_final_states({'q2'})
word = Word([Symbol('a'), Symbol('b')])
print(accept(nfae, word))  

ejemplo de accept
True


## Empty

In [105]:
#empty devuelve el epsilon automata que acepta el lenguaje vacio
def empty() -> NFAE:
    nfae = NFAE()
    nfae.set_delta({
        'q0': {}
    })
    nfae.set_init_state('q0')
    nfae.set_final_states(set())
    return nfae
#ejemplo de uso
print("ejemplo de empty")
nfae = empty()
word = Word([Symbol('a'), Symbol('b')])
print(accept(nfae, word))  

ejemplo de empty
False


## Epsilon

In [106]:
#epsilon devuelve el epsilon automata que acepta el lenguaje que solo contiene a epsilon
def epsilon() -> NFAE:
    nfae = NFAE()
    nfae.set_delta({
        'q0': {
            CharWithEpsilon(None): {'q1'}
        }
    })
    nfae.set_init_state('q0')
    nfae.set_final_states({'q1'})
    return nfae
#ejemplo de uso
print("ejemplo de epsilon")
nfae = epsilon()
word = Word([Symbol('ε')])
print(accept(nfae, word)) 

ejemplo de epsilon
False


## Symbol

In [107]:
#symbol esta recibe un simbolo y regresa el epsilon automata que acepta el lenguaje que acepta a la cadena a
def symbol(symbol: Symbol) -> NFAE:
    nfae = NFAE()
    nfae.set_delta({
        'q0': {
            CharWithEpsilon(str(symbol)): {'q1'}
        }
    })
    nfae.set_init_state('q0')
    nfae.set_final_states({'q1'})
    return nfae
#ejemplo de uso
print("ejemplo de symbol")
nfae = symbol(Symbol('a'))
word = Word([Symbol('a')])
print(accept(nfae, word)) 

ejemplo de symbol
True


## Union

In [108]:
#union recibe dos epsilon automatas y regresa la union utilizando trnasiciones epsilon
def union(nfae1: NFAE, nfae2: NFAE) -> NFAE:
    nfae = NFAE()
    nfae.set_init_state('q0')
    nfae.set_final_states(nfae2.final_states|nfae1.final_states)
    nfae.set_states(nfae1.states + nfae2.states)
    nfae.set_delta({
        'q0': {
            CharWithEpsilon(None): {nfae1.init_state, nfae2.init_state}
        }
    })
    for estado, transiciones in nfae1.delta.items():
        nfae.delta[estado] = transiciones.copy()
    for estado, transiciones in nfae2.delta.items():
        if estado in nfae.delta:
            nfae.delta[estado].update(transiciones)
        else:
            nfae.delta[estado] = transiciones
    return nfae
#ejemplo de uso
print("ejemplo de union")
nfae1 = symbol(Symbol('a'))
nfae2 = symbol(Symbol('b'))
nfae = union(nfae1, nfae2)
print(nfae.delta.items())  # Salida: dict_items([('q0', {ε: {'q1', 'q2'}}), ('q1', {'a': {'q2'}}), ('q2', {'b': {'q3'}})])
print(nfae.delta.get('q0'))  # Salida: {ε: {'q1', 'q2'}}
word = Word([Symbol('a')])
print("salida")
print(accept(nfae, word))  # Salida: True
#nota de los primeros outputs salen las direcciones de memoria de los estados, pero en el output final se muestra el estado en si

ejemplo de union
dict_items([('q0', {<__main__.CharWithEpsilon object at 0x0000015F801ADDD0>: {'q1'}, <__main__.CharWithEpsilon object at 0x0000015FFFBC8190>: {'q1'}})])
{<__main__.CharWithEpsilon object at 0x0000015F801ADDD0>: {'q1'}, <__main__.CharWithEpsilon object at 0x0000015FFFBC8190>: {'q1'}}
salida
True


## Concat

In [109]:
#concat recibe dos epsilon automatas y regresa la concatenacion de ambos utilizando transiciones epsilon
def concat(nfae1: NFAE, nfae2: NFAE) -> NFAE:
    nfae = NFAE()
    nfae.set_init_state(nfae1.init_state)
    nfae.set_final_states(nfae2.final_states)
    nfae.set_states(nfae1.states + nfae2.states)

    for estado, transiciones in nfae1.delta.items():
        nfae.delta[estado] = transiciones.copy()

    for estado, transiciones in nfae2.delta.items():
        if estado in nfae.delta:
            nfae.delta[estado].update(transiciones)
        else:
            nfae.delta[estado] = transiciones.copy()
            
    for estado in nfae1.final_states:
        if estado not in nfae.delta:
            nfae.delta[estado] = {}
        if CharWithEpsilon(None) in nfae.delta[estado]:
            nfae.delta[estado][CharWithEpsilon(None)].add(nfae2.init_state)
        else:
            nfae.delta[estado][CharWithEpsilon(None)] = {nfae2.init_state}
    
    return nfae

# Ejemplo de uso
print("ejemplo de concat")
nfae1 = symbol(Symbol('a'))
nfae2 = symbol(Symbol('b'))
nfae = concat(nfae1, nfae2)
word = Word([Symbol('a'), Symbol('b')])
print(accept(nfae, word))  

ejemplo de concat
True


## Kleene

In [110]:
#kleene recibe un epsilon automata y regresa la cerradura de kleene de ese automata
def kleene(nfae: NFAE) -> NFAE:
    nfae_kleene = NFAE()
    nfae_kleene.set_init_state('q0')
    nfae_kleene.set_final_states(nfae.final_states | {'q0'})
    nfae_kleene.set_states(nfae.states + ['q0'])
    
    nfae_kleene.set_delta({
        'q0': {
            CharWithEpsilon(None): {nfae.init_state}
        }
    })
    
    for estado, transiciones in nfae.delta.items():
        nfae_kleene.delta[estado] = transiciones.copy()
    
    for estado in nfae.final_states:
        if estado not in nfae_kleene.delta:
            nfae_kleene.delta[estado] = {}
        if CharWithEpsilon(None) in nfae_kleene.delta[estado]:
            nfae_kleene.delta[estado][CharWithEpsilon(None)].add(nfae.init_state)
        else:
            nfae_kleene.delta[estado][CharWithEpsilon(None)] = {nfae.init_state}

        nfae_kleene.delta[estado][CharWithEpsilon(None)].add('q0')
    
    return nfae_kleene

#ejemplo de uso
nfae = symbol(Symbol('a'))
nfae = kleene(nfae)
word = Word([Symbol('a'), Symbol('a')])
print(accept(nfae, word))  

True


## ToEAFN

In [111]:
#toEAFN Esta funcion recibe una expresion regular e y devuelve el $\epsilon-AFN, E$ tal que $L(e)=L(E)$
def ToEAFN(e: RE) -> NFAE:
    if isinstance(e, Empty):
        return empty()
    elif isinstance(e, Epsilon):
        return epsilon()
    elif isinstance(e, Sym):
        return symbol(Symbol(e.char))
    elif isinstance(e, Conc):
        nfae1 = ToEAFN(e.e1)
        nfae2 = ToEAFN(e.e2)
        return concat(nfae1, nfae2)
    elif isinstance(e, Union):
        nfae1 = ToEAFN(e.e1)
        nfae2 = ToEAFN(e.e2)
        return union(nfae1, nfae2)
    elif isinstance(e, Kleene):
        nfae = ToEAFN(e.e)
        return kleene(nfae)
    else:
        raise ValueError("No se reconoce esa expresion regular")
#ejemplo de uso    
regex =Kleene(Union(Sym('a'), Sym('b')))
nfae = ToEAFN(regex)
palabra1=Word([Symbol('a'),Symbol('a'),Symbol('a')])
palabra2=Word([Symbol('b'),Symbol('b'),Symbol('b')])
print(accept(nfae, palabra1))  
print(accept(nfae, palabra2))  

True
True
